# Pregnacy Outcome Prediction using LinearSVC

In [41]:
#Importing necessary libraries

from matplotlib import style
import numpy as np
style.use('ggplot')

from sklearn.svm import LinearSVC

from sklearn import preprocessing

import pandas as pd

In [42]:
#Removing unnecessary columns and cleaning the data for further use in modeling

def scrubData(df):
    
    #Assuming that 1 corresponds to still birth
    dfStill = df[df.outcome_pregnancy == 1]
    print(dfStill.shape)
    #Assuming that 2 corresponds to live birth
    dfLve = df[df.outcome_pregnancy == 2]
    dfSpontArb = df[df.outcome_pregnancy == 4 ]
    
    totalStill = len(dfStill.index)
    totalSpontArb = len(dfSpontArb.index)
    
    totalOK = len(dfLve.index)
    
    dfStillAndLive = dfStill.append(dfLve)
    dAll = dfStillAndLive.append(dfSpontArb)
    print(dAll.shape)
    df = dAll
    
    df = df.sample(frac=1.0)
    
    #Removing unnecessary columns
    df.drop(['w_id','hl_id', 'client_w_id','state','district'], 1, inplace=True)
    df.drop(['psu_id', 'house_no','house_hold_no','year_of_intr', 'month_of_intr', 'date_of_intr'], 1, inplace=True)    
    df.drop(['other_int_code','identifcation_code', 'w_expall_status','w_status','twsi_id', 'client_twsi_id'], 1, inplace=True)
    df.drop(['fid','hh_id', 'client_hh_id','member_identity','father_serial_no', 'mother_serial_no'], 1, inplace=True)
    df.drop(['client_hl_id','building_no', 'hl_expall_status','sn'], 1, inplace=True)
    df.drop(['headname','ever_born', 'wt'], 1, inplace=True)
    df.drop(['fidx','as', 'as_binned'], 1, inplace=True)
    df.drop(['fidh','cdoi', 'anym','catage1','respondentname', 'rtelephoneno'], 1, inplace=True)
    df.drop(['healthscheme_1','healthscheme_2'], 1, inplace=True)
    df.drop(['new_born_alive_female','new_born_alive_male', 'new_born_alive_total','new_surviving_female'], 1, inplace=True)  
    df.drop(['new_surviving_male','new_surviving_total'], 1, inplace=True) 
    df.drop(['isdeadmigrated'], 1, inplace=True)
    df.drop(['isnewrecord','recordupdatedcount', 'recordupdatedcount','schedule_id','year', 'id'], 1, inplace=True)
    df.drop(['date_of_marriage', 'month_of_marriage', 'year_of_marriage'],1, inplace=True)
    df.drop(['year_of_birth', 'month_of_birth', 'date_of_birth'],1, inplace=True)
    df.drop(['compensation_after_ster', 'received_compensation_after_ster', 'received_compensation_ster_rs'],1, inplace=True)
    df.drop(['is_tubectomy','delivered_any_baby','born_alive_female','born_alive_male','born_alive_total'],1, inplace=True)
    df.drop(['surviving_female','surviving_male','surviving_total'],1, inplace=True)
    df.drop(['last_preg_no','previous_last_preg_no','second_last_preg_no', 'third_last_preg_no'],1, inplace=True)
    df.drop(['edt','occupation','marital', 'modern', 'traditional'],1, inplace=True)
    df.drop(['recordstatus'],1, inplace=True)
    df.drop(['hh_serial_no'],1, inplace=True)
    df.drop(['sex'],1, inplace=True)
    df.drop(['religion'],1, inplace=True) 
    df.drop(['currently_attending_school'],1, inplace=True) 
    df.drop(['reason_for_not_attending_school'],1, inplace=True)
    df.drop(['highest_qualification'],1, inplace=True)
    df.drop(['serial_no'],1, inplace=True)
    df.drop(['disability_status'],1, inplace=True)
    df.drop(['occupation_status'],1, inplace=True)
    df.drop(['house_status'],1, inplace=True)
    df.drop(['householdstatus'],1, inplace=True)
    df.drop(['isheadchanged'],1, inplace=True)
    df.drop(['land_possessed'],1, inplace=True)
    df.drop(['cart'],1, inplace=True)
    df.drop(['current_mar_status'],1, inplace=True)
    df.drop(['counselled_for_menstrual_hyg'],1, inplace=True)
    df.drop(['aware_abt_haf'],1, inplace=True)
    df.drop(['aware_abt_ort_ors'],1, inplace=True)
    df.drop(['aware_abt_ort_ors_zinc'],1, inplace=True)
    df.drop(['aware_abt_danger_signs_new_born'],1, inplace=True)  
    df.drop(['residancial_status'],1, inplace=True)  
    df.drop(['iscoveredbyhealthscheme'],1, inplace=True)  
    df.drop(['housestatus'],1, inplace=True)  
    df.drop(['health_prob_afters_fp_use'],1, inplace=True)  
    df.drop(['is_husband_living_with_you'],1, inplace=True)  
    df.drop(['months_of_preg_first_anc'],1, inplace=True)
    df.drop(['age_at_first_conception'],1, inplace=True)
    df.drop(['regular_treatment_source'],1, inplace=True) 
    df.drop(['regular_treatment'],1, inplace=True) 
    df.drop(['diagnosis_source'],1, inplace=True)
    df.drop(['diagnosed_for'],1, inplace=True)
    df.drop(['sought_medical_care'],1, inplace=True)
    df.drop(['symptoms_pertaining_illness'],1, inplace=True)
    df.drop(['treatment_source'],1, inplace=True)
    df.drop(['illness_type'],1, inplace=True)
    df.drop(['injury_treatment_type'],1, inplace=True)
    df.drop(['currently_dead_or_out_migrated'],1, inplace=True)
    df.drop(['twsi_expall_status'],1, inplace=True)
    df.drop(['currently_widow'],1, inplace=True)
    
    
    df.replace('', np.nan)
    
    #Converting to numeric form and filling 0 in place of NaN values
    df.apply(pd.to_numeric, errors='coerce')
    df.fillna(0, inplace=True)
    
    
    df = df[df.result_of_interview == 1]
    df.drop(['result_of_interview'],1, inplace=True)
    
    print(df.shape)
    return df

In [43]:
#Importing the larger dataset on which predictions are to be made 

df2 = pd.read_csv('5.csv')

C:\Users\Rhythm\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1,12,14,28,29,30,31,32,33,36,72,73,76,78,80,81,98,99,117,124,147,170,178,185,186,193,194) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [44]:
df2

w_id hl_id  client_w_id  state  district  rural  stratum_code  \
0        NaN   NaN          NaN      5        13      1             2   
1        NaN   NaN          NaN      5        13      1             2   
2        NaN   NaN          NaN      5        13      1             2   
3        NaN   NaN          NaN      5        13      1             2   
4        NaN   NaN          NaN      5        13      1             2   
...      ...   ...          ...    ...       ...    ...           ...   
941126   NaN   NaN          NaN      5         2      2             0   
941127   NaN   NaN          NaN      5         2      2             0   
941128   NaN   NaN          NaN      5         2      2             0   
941129   NaN   NaN          NaN      5         2      2             0   
941130   NaN   NaN          NaN      5         2      2             0   

           psu_id  house_no  house_hold_no  ...  anym  respondentname  \
0       100426933         4              1  ...   NaN             NaN   
1       100426778         8              1  ...   NaN             NaN   
2       100427331        11              1  ...   NaN             NaN   
3       100427695        20              1  ...   NaN             NaN   
4       100427501        21              1  ...   NaN             NaN   
...           ...       ...            ...  ...   ...             ...   
941126  100505198        36              1  ...   NaN             NaN   
941127  100505166        43              1  ...   NaN             NaN   
941128  100505164        44              1  ...   NaN             NaN   
941129  100505183        46              1  ...   NaN             NaN   
941130  100505140        51              1  ...   NaN             NaN   

       rtelephoneno  isnewrecord recordupdatedcount  recordstatus  \
0               NaN          NaN                1.0           2.0   
1               NaN          NaN                1.0           2.0   
2               NaN          NaN                1.0           2.0   
3               NaN          NaN                1.0           2.0   
4               NaN          NaN                1.0           2.0   
...             ...          ...                ...           ...   
941126          NaN          NaN                1.0           2.0   
941127          NaN          NaN                1.0           2.0   
941128          NaN          1.0                1.0           2.0   
941129          NaN          NaN                1.0           2.0   
941130          NaN          NaN                1.0           2.0   

        schedule_id  year       id  v204  
0                 2     3  1490121   NaN  
1                 2     3  1490122   NaN  
2                 2     3  1490123   NaN  
3                 2     3  1490124   NaN  
4                 2     3  1490125   NaN  
...             ...   ...      ...   ...  
941126            2     2  9023060   NaN  
941127            2     2  9023061   NaN  
941128            2     2  9023062   NaN  
941129            2     2  9023063   NaN  
941130            2     2  9023064   NaN  

[941131 rows x 202 columns]

In [45]:
#Generating a smaller dataset from the larger one. On this smaller dataset, the model will be trained.

alpha = df2.sample(frac = 0.02656378)

In [46]:
alpha

w_id   hl_id  client_w_id  state  district  rural  stratum_code  \
107193      NaN     NaN          NaN      5         6      1             2   
702728   6017.0   12665       1971.0      5         2      1             1   
510706      NaN                  NaN      5         9      1             1   
859830      NaN     NaN          NaN      5         6      1             1   
459214      NaN     NaN          NaN      5         7      1             1   
...         ...     ...          ...    ...       ...    ...           ...   
824498      NaN     NaN          NaN      5         1      1             1   
668839   2794.0    5395       1352.0      5        11      1             1   
82283       NaN     NaN          NaN      5         6      1             1   
125887      NaN     NaN          NaN      5         4      1             1   
751921  67846.0  134017       3579.0      5         3      1             1   

           psu_id  house_no  house_hold_no  ...  anym  respondentname  \
107193  100415405       137              2  ...   NaN             NaN   
702728  100505825       151              2  ...   NaN             NaN   
510706  100778980       173              1  ...   NaN                   
859830  100360378       180              1  ...   NaN             NaN   
459214  100688772       124              1  ...   NaN             NaN   
...           ...       ...            ...  ...   ...             ...   
824498  100013775       319              1  ...   NaN             NaN   
668839  100868355       115              1  ...   NaN             NaN   
82283   100355248        65              1  ...   NaN             NaN   
125887  100137702        86              1  ...   NaN             NaN   
751921  100578266       223              2  ...   NaN             NaN   

       rtelephoneno  isnewrecord recordupdatedcount  recordstatus  \
107193          NaN          NaN                1.0           2.0   
702728          NaN          NaN                NaN           NaN   
510706                       NaN                1.0           2.0   
859830          NaN          NaN                1.0           2.0   
459214          NaN          NaN                1.0           2.0   
...             ...          ...                ...           ...   
824498          NaN          NaN                1.0           2.0   
668839          NaN          NaN                NaN           NaN   
82283           NaN          NaN                1.0           2.0   
125887          NaN          1.0                1.0           2.0   
751921          NaN          NaN                NaN           NaN   

        schedule_id  year       id  v204  
107193            2     3  1374103   NaN  
702728            2     1  8783782   NaN  
510706            2     2  9214403   NaN  
859830            2     2  9325897   NaN  
459214            2     2  9170709   NaN  
...             ...   ...      ...   ...  
824498            2     2  9234627   NaN  
668839            2     1  8733868   NaN  
82283             2     3  1351575   NaN  
125887            2     3  1264319   NaN  
751921            2     1  8811338   NaN  

[25000 rows x 202 columns]

In [47]:
#Running the data cleaning function on the smaller sample dataset

df = scrubData(alpha)

(3661, 202)
(24012, 202)
(23689, 82)


In [48]:
df

rural  stratum_code  age  marital_status  \
362723      1             1   43               3   
539680      2             0   45               3   
913503      2             0   25               3   
337901      1             1   48               3   
655236      1             2   48               3   
...       ...           ...  ...             ...   
100699      1             1   35               3   
34499       1             2   19               3   
54599       2             0   41               3   
923492      1             2   23               3   
321424      1             1   31               7   

        mother_age_when_baby_was_born  outcome_pregnancy  is_vasectomy  \
362723                           21.0                2.0           1.0   
539680                           26.0                2.0           1.0   
913503                           16.0                2.0           1.0   
337901                           17.0                2.0           1.0   
655236                           30.0                2.0           1.0   
...                               ...                ...           ...   
100699                           30.0                2.0           1.0   
34499                             0.0                1.0           1.0   
54599                            24.0                2.0           1.0   
923492                           21.0                1.0           1.0   
321424                           19.0                2.0           0.0   

        is_copper_t  is_pills_daily  is_piils_weekly  ...  is_sewing_machine  \
362723          2.0             1.0              1.0  ...                2.0   
539680          1.0             1.0              1.0  ...                2.0   
913503          1.0             1.0              1.0  ...                2.0   
337901          2.0             2.0              2.0  ...                1.0   
655236          1.0             1.0              1.0  ...                2.0   
...             ...             ...              ...  ...                ...   
100699          1.0             1.0              1.0  ...                0.0   
34499           1.0             1.0              1.0  ...                0.0   
54599           1.0             1.0              1.0  ...                1.0   
923492          1.0             1.0              1.0  ...                1.0   
321424          0.0             0.0              0.0  ...                2.0   

        is_bicycle  is_scooter  is_car  is_tractor  is_water_pump  \
362723         2.0         2.0     2.0         2.0            2.0   
539680         1.0         2.0     2.0         2.0            2.0   
913503         1.0         1.0     2.0         2.0            2.0   
337901         2.0         2.0     1.0         2.0            2.0   
655236         2.0         2.0     2.0         2.0            2.0   
...            ...         ...     ...         ...            ...   
100699         0.0         0.0     0.0         0.0            0.0   
34499          0.0         0.0     0.0         0.0            0.0   
54599          1.0         1.0     2.0         2.0            2.0   
923492         2.0         1.0     2.0         2.0            2.0   
321424         2.0         2.0     2.0         2.0            2.0   

        ever_conceived  no_of_times_conceived  is_injectable_contraceptive  \
362723             1.0                    5.0                          2.0   
539680             1.0                    5.0                          1.0   
913503             0.0                    0.0                          0.0   
337901             1.0                    4.0                          2.0   
655236             0.0                    0.0                          0.0   
...                ...                    ...                          ...   
100699             1.0                    1.0                          2.0   
34499              1.0                    1.0                          1.0   
54

In [57]:
#Features for model training

X = np.array(df.drop(['outcome_pregnancy'], 1))
print(X.shape)

(23689, 81)


In [58]:
for col in df.columns:
    print(col, df[col].unique())

rural [1 2]
stratum_code [1 0 2]
age [43 45 25 48 21 27 39 20 32 44 42 22 29 31 36 26 28 47 41 24 35 19 46 38
 30 37 33 40 34 23 49 18 17 16 15]
marital_status [3 5 6 7 4]
mother_age_when_baby_was_born [21. 26. 16. 17. 30. 27. 18.  0. 22. 28. 23. 19. 24. 20. 25. 15. 31. 14.
 13. 32. 12. 34. 29. 33. 35. 40. 37. 36. 39.]
outcome_pregnancy [2. 1.]
is_vasectomy [1. 0. 2.]
is_copper_t [2. 1. 0.]
is_pills_daily [1. 2. 0.]
is_piils_weekly [1. 2. 0.]
is_emergency_contraceptive [1. 2. 0.]
is_condom [1. 2. 0.]
is_moder_methods [2. 1. 0.]
is_contraceptive [2. 1. 0.]
is_periodic_abstinence [2. 1. 0.]
is_withdrawal [1. 2. 0.]
is_amenorrahoea [2. 1. 0.]
is_other_traditional_method [2. 1. 0.]
is_currently_pregnant [2. 1. 0. 3.]
pregnant_month [0. 7. 8. 9. 6. 1. 5. 4. 2. 3.]
is_anc_registered [0. 1. 2.]
willing_to_get_pregnant [0. 1. 2. 4. 3.]
is_currently_menstruating [1. 4. 5. 0. 2. 3. 6.]
when_you_bcome_mother_last_time [1. 0. 2. 4. 3.]
is_any_fp_methos_used [1. 0. 2.]
fp_method_used [ 1.  0.  3.  

In [59]:
df.iloc[:,40] = df.iloc[:,40].replace(' ',0)
df.iloc[:,41] = df.iloc[:,41].replace(' ',0)
df.iloc[:,58] = df.iloc[:,58].replace(' ',0)

In [60]:
df.iloc[:,40]

362723    0
539680    0
913503    0
337901    0
655236    0
         ..
100699    0
34499     0
54599     0
923492    0
321424    0
Name: during_pregnancy, Length: 23689, dtype: object

In [61]:
#Standardizing the feature dataset
X = preprocessing.scale(X)

#Prediction variable
y = np.array(df['outcome_pregnancy'])

In [62]:
#Generating the object for LinearSVC method and fitting it on the smaller sample dataset for training

clf = LinearSVC(max_iter=90000)
clf.fit(X,y)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=90000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [63]:
correct = 0

In [64]:
#Prediction on the training dataset

for i in range(len(X)):
    predict_me = np.array(X[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = clf.predict(predict_me)
   
    if prediction[0] == y[i]:
        correct += 1

In [65]:
#Accuracy check

print("result: ", correct/len(X))

result:  0.9101270631938875


In [66]:
#Running the cleaning function on the actual larger dataset

df2 = scrubData(df2)

(135796, 202)
(903180, 202)
(891589, 82)


In [71]:
#Feature dataset

X2 = np.array(df2.drop(['outcome_pregnancy'], 1))
print(X2.shape)

(891589, 81)


In [72]:
for col in df2.columns:
    print(col, df2[col].unique())

rural [1 2]
stratum_code [1 0 2]
age [19 30 21 45 28 37 46 24 36 42 25 49 27 32 23 31 26 38 35 41 40 29 48 39
 47 20 43 34 33 22 44 17 18 16 15]
marital_status [3 5 4 7 6 2]
mother_age_when_baby_was_born [19. 21. 20. 17.  0. 22. 26. 24. 23. 30. 15. 18. 28. 27. 25. 16. 29. 31.
 14. 41. 34. 35. 32. 33. 36. 12. 40. 13. 37. 42. 38. 43. 39. 44. 48. 45.
 47. 46. 49.]
outcome_pregnancy [1. 2.]
is_vasectomy [1. 0. 2.]
is_copper_t [1. 2. 0.]
is_pills_daily [1. 2. 0.]
is_piils_weekly [1. 2. 0.]
is_emergency_contraceptive [1. 2. 0.]
is_condom [1. 2. 0.]
is_moder_methods [2. 1. 0.]
is_contraceptive [1. 2. 0.]
is_periodic_abstinence [1. 2. 0.]
is_withdrawal [1. 2. 0.]
is_amenorrahoea [1. 2. 0.]
is_other_traditional_method [2. 1. 0.]
is_currently_pregnant [2. 0. 3. 1.]
pregnant_month [0. 8. 1. 6. 4. 2. 7. 5. 3. 9.]
is_anc_registered [0. 2. 1.]
willing_to_get_pregnant [0. 1. 2. 4. 3.]
is_currently_menstruating [2. 1. 0. 4. 5. 3. 6.]
when_you_bcome_mother_last_time [1. 2. 0. 4. 3.]
is_any_fp_methos_us

In [73]:
df2.iloc[:,40] = df2.iloc[:,40].replace(' ',0)
df2.iloc[:,41] = df2.iloc[:,41].replace(' ',0)
df2.iloc[:,58] = df2.iloc[:,58].replace(' ',0)

In [74]:
#Standardizing the feature dataset
X2 = preprocessing.scale(X2)

#Prediction variable
y2 = np.array(df2['outcome_pregnancy'])

In [75]:
correct = 0

In [76]:
#Prediction (Larger dataset)

for i in range(len(X2)):
    predict_me = np.array(X2[i].astype(float))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = clf.predict(predict_me)
   
    if prediction[0] == y2[i]:
        correct += 1

In [77]:
#Accuracy check

print("2nd result: ", correct/len(X2))

2nd result:  0.9106527783541519
